In [1]:
%pip install -q \
    fuzzywuzzy \
    easyocr \
    pandas \
    requests


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# seed reference items using api

# import from csv
import pandas as pd
import requests

referenceitems_df = pd.read_csv('referenceitems.csv')

def writeToEndpoint(endpoint, json):
    url = f'http://localhost:3000/api/{endpoint}'
    response = requests.post(url, json=json)
    if response.status_code == 200:
        print(f'Succes writing {endpoint}. Response:\n{response.text}')
    else:
        print(f'Error writing {endpoint}. Response:\n{response.text}')
    return response

for index, row in referenceitems_df.iterrows():
    response = writeToEndpoint('referenceItem', {
        "name": row['name'],
        "quantity": row['quantity'],
        "unitOfMeasure": row['unitofmeasure'],
        "price": row['price'],
        "pricePerWeight": row['priceperweight'],
        "referenceUrl": row['referenceurl']
    })
    new_referenceitem_id = response.json()['id']
    referenceitems_df.at[index, 'id'] = int(new_referenceitem_id)
referenceitems_df['id'] = referenceitems_df['id'].astype(int)
referenceitems_df.set_index('id', inplace=True)
# referenceitems_df = referenceitems_df[['id', 'name', 'quantity', 'unitofmeasure', 'price', 'priceperweight', 'referenceurl']]
referenceitems_df

# test
# writeReferenceItem({
#     "name": "Sample item",
#     "quantity": 10,
#     "unitOfMeasure": "g",
#     "price": 100,
#     "pricePerWeight": 10,
#     "referenceUrl": "test.com"
# })

# import from queryclassification.csv to panda dataframe
queryclassification_df = pd.read_csv('queryclassification.csv')
queryclassification_df.reset_index(drop=True, inplace=True)
queryclassification_df['id'] = queryclassification_df.index
queryclassification_df.set_index('id', inplace=True)
queryclassification_df

temp_referenceItem_map = referenceitems_df.copy()
temp_referenceItem_map['id'] = temp_referenceItem_map.index
temp_referenceItem_map

queryclassification_df['referenceItemId'] = queryclassification_df['referenceItem'].map(temp_referenceItem_map.set_index('name')['id'].to_dict())
try:
    queryclassification_df['referenceItemId'] = queryclassification_df['referenceItemId'].astype(int)
except:
    print('Failed to convert referenceItemId to int')
    print(queryclassification_df[queryclassification_df['referenceItemId'].isnull()])
queryclassification_df.drop(columns=['referenceItem'], inplace=True)
queryclassification_df

Succes writing referenceItem. Response:
{"id":1,"name":"BBQ Sauce","quantity":455,"unitOfMeasure":"g","price":2.47,"pricePerWeight":0.005429,"referenceUrl":"https://walmart.ca/#missing","createdAt":"2024-08-02T00:48:20.296Z","updatedAt":"2024-08-02T00:48:20.296Z"}
Succes writing referenceItem. Response:
{"id":2,"name":"Beans","quantity":540,"unitOfMeasure":"g","price":1.37,"pricePerWeight":0.002537,"referenceUrl":"https://walmart.ca/#missing","createdAt":"2024-08-02T00:48:20.318Z","updatedAt":"2024-08-02T00:48:20.318Z"}
Succes writing referenceItem. Response:
{"id":3,"name":"Bouillon cubes","quantity":80,"unitOfMeasure":"g","price":1.14,"pricePerWeight":0.01425,"referenceUrl":"https://walmart.ca/#missing","createdAt":"2024-08-02T00:48:20.327Z","updatedAt":"2024-08-02T00:48:20.327Z"}
Succes writing referenceItem. Response:
{"id":4,"name":"Candy","quantity":150,"unitOfMeasure":"g","price":1,"pricePerWeight":0.006667,"referenceUrl":"https://walmart.ca/#missing","createdAt":"2024-08-02T00:

,query,referenceItemId
id,,
0,3x KIND Oats & Honey Granola with Toasted Coco...,14
1,Annie's Homegrown Gluten-Free Rice Pasta & Che...,19
2,Annie's Homegrown Gluten-Free Rice Pasta White...,19
3,Annies rice pasta and white cheddar,19
4,Baldstreet all beef frankfurters,40
...,...,...
169,Mac & Cheese,19
170,Caramel Chocolate Chip Cookie Bar,34
171,Purest White Bread,32


In [8]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re
import easyocr
import pandas as pd
import shutil
import numpy as np

reader = easyocr.Reader(['en']) # this needs to run only once to load the model into memory

def ocrImage(image_path):
    image_id = 1 # fake image id
    receipt_strings = reader.readtext(image_path)

    # guard clause
    if len(receipt_strings) < 1:
        return # no text found
    
    # ... do the things

    # "receiptTexts": [
    #                     {
    #                         "id": "123",
    #                         "text": "Item 1",
    #                         "boundingBox": "{'x':0,'y':0,'width':100,'height':20}", 
    #                         "imageFileId": "123"
    #                     },
    #                     {
    #                         "id": "124",
    #                         "text": "Item 2",
    #                         "boundingBox": "{'x':0,'y':0,'width':100,'height':20}", 
    #                         "imageFileId": "123"
    #                     }
    #                 ]

    # transformation
    receipt_strings_df = pd.DataFrame(receipt_strings, columns=['boundingBox', 'text', 'confidence'])
    receipt_strings_df.drop(columns=['confidence'], inplace=True)
    receipt_strings_df['imageFileId'] = image_id
    # receipt_strings_df['imageFileId'] = receipt_strings_df['imageFileId'].astype(int)

    # push to API
    for index, row in receipt_strings_df.iterrows():
        # print(row['text'])
        # print(row['boundingBox'])
        # print(row['imageFileId'])
        boundingBox_str = [[int(value) for value in sublist] for sublist in row['boundingBox']]
        response = writeToEndpoint('receiptText', {
            'text': str(row['text']), 
            'boundingBox': str(boundingBox_str), 
            'imageFileId': int(row['imageFileId'])
        })
        print(response)
    # return

choices_dict = queryclassification_df[['query']].to_dict()['query']
def get_reference_item_id(best_match):
    return queryclassification_df[queryclassification_df['query'] == best_match].iloc[0]['referenceItemId']

# consider global API function: query or mutation
# async def api_call(queryormutation, variable_values):
    # try:
    #     data = await client.execute_async(queryormutation, variable_values=variable_values)
    #     print(data)
    # except Exception as e:
    #     print(f"Query failed: {e}")

def fetch_receipt(receiptID):
# TODO: implement
# async def fetch_receipt_details():
    # try:
    #     data = await client.execute_async(query, variable_values={"receiptID": receiptID})
    #     print(data)
    # except Exception as e:
    #     print(f"Query failed: {e}")
    return {
        "data": {
            "receipt": {
                # "vendor": {
                #     "id": "123",
                #     "name": "Vendor Name"
                # },
                # "images": [
                #     {
                #         "id": "123",
                #         "filename": "IMG_5903.jpg"
                #     }
                # ],
                # "receiptTexts": [
                #                     {
                #                         "id": "123",
                #                         "text": "Item 1",
                #                         "boundingBox": "{'x':0,'y':0,'width':100,'height':20}", 
                #                         "imageFileId": "123"
                #                     },
                #                     {
                #                         "id": "124",
                #                         "text": "Item 2",
                #                         "boundingBox": "{'x':0,'y':0,'width':100,'height':20}", 
                #                         "imageFileId": "123"
                #                     }
                #                 ]
            }
        }
    }

def parseTextForEligibleExpenses(receipt_id):
    receipt = fetch_receipt(receipt_id)
    receipt_texts = receipt['data']['receipt']['receiptTexts']
    threshold = 80

    # TODO: some clean up before look up
        # trim numbers in the beginning of the string, drop too short alphanumeric strings
            # search_query = re.sub(r'^\d{1,10}', '', search_query)
            # # if len(search_query) > 2:
        # if search_query contains at least one digit or letter
            # if any(char.isalnum() for char in search_query):
        # regex out 1 to 10 digits in the beginning of the string for sku codes in store receipts
    
    # for now choices are static
    # choices = get_choices() # receipt_text, receipt_vendor)
    # TODO: clean up choices too 
        # avoid strict substring matches
            # len(search_query)/len(best_match) < 0.4
        # here I could also optimize for a given vendor if available
            # receipt_vendor = receipt['data']['receipt']['vendor']['name']

    for receipt_text in receipt_texts:
        receipt_text_id = receipt_text['id']
        receipt_text = receipt_text['text']
        _, best_score, best_match_id = process.extractOne(receipt_text, choices_dict, scorer=fuzz.token_set_ratio)
        if best_score >= threshold:
            # found a decent match
            # get corresponding referenceItem 
            reference_item_id = choices[best_match_id]['referenceItemId']
            # create eligible expense
            writeToEndpoint('expense',{
                    'receiptTextId': receipt_text_id,
                    'referenceItemId': reference_item_id,
                    'receiptId': receipt_id,
                    'priceEach': 0, # TODO: find price
                    'quantiy': 1, # TODO: find quantity
                }
            )

def upload_image(image_path):
    # fake upload, i.e. copy to frontend folder
    destination_folder = "../receipts-app/public/uploads/"
    shutil.copy(image_path, destination_folder)
    image_url = destination_folder + image_path.split("/")[-1]
    receipt_id = writeToEndpoint('receipt', {}).json()['id']
    response = writeToEndpoint('imageFile', {
        "url": image_url, 
        "receiptId": receipt_id,
        })
    return response
# test
# upload_image('./receipts/IMG_4553.jpg')

# TODO: batch multiple images in array
def main_single_receipt(image_path):
    # image = crop_image(image_path) # skipping this for now
    image_id = upload_image(image_path).json()['id']
    receiptTexts = ocrImage(image_path)
    # print('receiptTexts:')
    # print(receiptTexts)
    # ... need to match with reference items
    # createReceipt(receiptTexts)
    return image_id # receiptID

main_single_receipt('./receipts/IMG_4553.jpg')

Succes writing receipt. Response:
{"id":5,"createdAt":"2024-08-02T00:57:18.485Z","updatedAt":"2024-08-02T00:57:18.485Z"}
Succes writing imageFile. Response:
{"id":5,"url":"../receipts-app/public/uploads/IMG_4553.jpg","receiptId":5,"createdAt":"2024-08-02T00:57:18.490Z","updatedAt":"2024-08-02T00:57:18.490Z"}


TypeError: Object of type int32 is not JSON serializable